In [1]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import numpy as np

In [2]:
!bash download_voc.sh

--2021-04-04 11:03:01--  http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Resolving www.eecs.berkeley.edu (www.eecs.berkeley.edu)... 23.185.0.1, 2620:12a:8001::1, 2620:12a:8000::1
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-04-04 11:03:01--  https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-04-04 11:03:01--  https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic

In [4]:
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/model.py'

--2021-04-04 11:08:22--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3648 (3.6K) [text/plain]
Saving to: ‘extract_weights.py.1’

extract_weights.py. 100%[===================>]   3.56K  --.-KB/s    in 0s      

2021-04-04 11:08:22 (63.3 MB/s) - ‘extract_weights.py.1’ saved [3648/3648]

--2021-04-04 11:08:22--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [5]:
from VOCGen import get_voc_datagen
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Mean, BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy
from gan_model import AdverGAN
import os
from metrics import mean_iou

In [6]:
# 20 classes from PASCAL VOC and 1 background
N_CLASS = 20 + 1 
BATCH_SIZE = 12
img_target_size = (320, 320)

In [7]:
img_preproc = lambda x: x / 255.
tr_gen, val_gen = get_voc_datagen(N_CLASS, BATCH_SIZE, img_preproc, None, img_target_size)

In [8]:
g_iou_tracker = Mean(name='iou')
gan_acc_tracker = Mean(name='gan_acc')
disc_acc_tracker = Mean(name='disc_acc')
iou_score = mean_iou
binary_entropy = BinaryCrossentropy()
gan_acc = BinaryAccuracy()
disc_acc = BinaryAccuracy()

In [9]:
for x,y in tr_gen:
  print(x.shape)
  print(y.shape)
  break

(12, 320, 320, 3)
(12, 320, 320, 21)


In [10]:
gan_model = AdverGAN(1e-2, img_target_size, N_CLASS)
g_opt = Adam(2.5e-4)
d_opt = Adam(1e-5)

166100992/166096424 [==============================] - 2s 0us/step


In [11]:
BEST_VAL_IOU = 0.0
MODEL_SAVE_DIR = os.path.join('drive', 'MyDrive')

def save_generator_discriminator_on_best(logs):
  global BEST_VAL_IOU
  if BEST_VAL_IOU < logs['val_gen_iou']:
    BEST_VAL_IOU = logs['val_gen_iou']
    gan_model.save_models(MODEL_SAVE_DIR)


saving_model_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: save_generator_discriminator_on_best(logs)
)

In [12]:
gan_model.compile(g_opt, d_opt, binary_entropy, gan_acc, disc_acc, categorical_crossentropy, iou_score, g_iou_tracker, gan_acc_tracker, disc_acc_tracker)

In [ ]:
gan_model.fit(tr_gen, epochs=40, validation_data=val_gen, callbacks=[saving_model_callback])

Epoch 1/40
709/709 [==============================] - 850s 1s/step - disc_fake_loss: 0.7084 - disc_real_loss: 0.6596 - gan_loss: 0.4213 - gen_iou: 0.5678 - g_loss: 0.4145 - disc_acc: 0.5468 - gan_acc: 0.5667 - val_disc_fake_loss: 0.7364 - val_disc_real_loss: 0.6499 - val_gan_loss: 0.0470 - val_gen_iou: 0.6211 - val_g_loss: 0.0404 - val_disc_acc: 0.6006 - val_gan_acc: 0.6424
Epoch 2/40
709/709 [==============================] - 825s 1s/step - disc_fake_loss: 0.6822 - disc_real_loss: 0.6357 - gan_loss: 0.3036 - gen_iou: 0.6583 - g_loss: 0.2964 - disc_acc: 0.6243 - gan_acc: 0.6420 - val_disc_fake_loss: 0.6961 - val_disc_real_loss: 0.6216 - val_gan_loss: 0.0921 - val_gen_iou: 0.6383 - val_g_loss: 0.0851 - val_disc_acc: 0.6478 - val_gan_acc: 0.6206
Epoch 3/40
709/709 [==============================] - 824s 1s/step - disc_fake_loss: 0.6484 - disc_real_loss: 0.6150 - gan_loss: 0.2401 - gen_iou: 0.6856 - g_loss: 0.2321 - disc_acc: 0.6576 - gan_acc: 0.6130 - val_disc_fake_loss: 0.7326 - val_dis